In [1]:
import numpy as np
import pandas as pd

for i in [np, pd]:
    print(i.__name__, i.__version__)

numpy 1.18.5
pandas 0.25.1


In [3]:
# 예제에서 사용할 데이터셋을 불러옵니다.
df_space = pd.read_csv('data/space_titanic.csv')
df_space.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Groupby

## by

**pd.DataFrame.groupby**

pd.DataFrame.groupby(self, by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False)

by:

    str - 그룹의 기준이 되는 컬럼명 또는 인덱스 이름/인덱스에 이름이 없다면, level을 사용 

    list - 그룹의 기준이 되는 컬럼명 또는 인덱스 이름 리스트

    Series, dict - DataFrame에 align (인덱스를 기준으로 join='left'할 때) DataFrame과, 대상 Series의 값 (예제에서 확인)
    
    np.ndarray - 동일한 행수의 np.ndarray와 순서에 대응하여 그룹의 기준
    
    function - 입력값을 DataFrame의 Index로 넘겨 주고, 반환값을 그룹의 기준

### Case 1

groupby HomePlanet으로 구분하여 RoomServce에 대한 평균을 구합니다.

In [4]:
df_space.groupby('HomePlanet')['RoomService'].mean()

HomePlanet
Earth     136.940209
Europa    145.231981
Mars      552.897272
Name: RoomService, dtype: float64

### Case 2

groupby HomePlanet과 Destination 으로 구분하여 RoomServce에 대한 평균을 구합니다.

In [5]:
df_space.groupby(['HomePlanet', 'Destination'])['RoomService'].mean()

HomePlanet  Destination  
Earth       55 Cancri e      143.392647
            PSO J318.5-22     71.185879
            TRAPPIST-1e      148.782293
Europa      55 Cancri e      145.276060
            PSO J318.5-22    143.111111
            TRAPPIST-1e      149.702055
Mars        55 Cancri e      596.553191
            PSO J318.5-22    369.765957
            TRAPPIST-1e      556.510021
Name: RoomService, dtype: float64

### Case 3

groupby - Cabin 을 / 로 구분했을 떄 첫번째 단어로 구분 Cabin이 결측일 때는 빈문자로 합니다. 이 때 Age의 평균을 구합니다.

In [6]:
df_space.groupby(
    df_space['Cabin'].str.split('/').apply(lambda x: x[0] if type(x) == list else '') # Series를 넘긴다
)['Age'].mean()

Cabin
     29.148718
A    35.161943
B    33.622715
C    34.886145
D    33.518201
E    30.068925
F    28.430495
G    24.062775
T    37.000000
Name: Age, dtype: float64

In [7]:
# 순서를 바꾸어도 index에 대응하므로 결과에는 영향이 없습니다.
df_space.sample(frac=1).groupby(
    df_space['Cabin'].str.split('/').apply(lambda x: x[0] if type(x) == list else '') # Series를 넘긴다
)['Age'].mean()

Cabin
     29.148718
A    35.161943
B    33.622715
C    34.886145
D    33.518201
E    30.068925
F    28.430495
G    24.062775
T    37.000000
Name: Age, dtype: float64

In [8]:
# ※ 주의 by로 넘어온 Series와 align하는 것이므로, 의도에 맞게 Series의 index가 구성이 되어 있는지를 확인해야 합니다.
# 아래와 같이 index가 일치하는 것에 대해서만 그룹화를 한다.

s_tmp = df_space['Cabin'].str.split('/').apply(lambda x: x[0] if type(x) == list else '')
s_tmp.index = -df_space.index # 음수로 바꾸어, 0 빼고 일치하지 않게 한다.
df_space.groupby(
    s_tmp
)['Age'].mean() # index가 일치한 0에 대한 값만 나옵니다.

Cabin
B    39.0
Name: Age, dtype: float64

In [9]:
# s_tmp를 align, join='left'로 해서 align 합니다.
_, s_ = df_space.align(s_tmp.rename('C2'), axis=0, join='left')
s_ # align 후, DataFrame의 index에 맞게 Series가 변환 됩니다.

0         B
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
8688    NaN
8689    NaN
8690    NaN
8691    NaN
8692    NaN
Name: C2, Length: 8693, dtype: object

In [10]:
df_space.groupby(s_)['Age'].mean() # 위와 같습니다.

C2
B    39.0
Name: Age, dtype: float64

In [11]:
# 만일 numpy.ndarray를 by에 입력한다면, DataFrame과 numpy array의 순서에 대응하여 groupby를 합니다.

s_tmp.values # numpy.ndarray로 반환

array(['B', 'F', 'A', ..., 'G', 'E', 'E'], dtype=object)

In [12]:
df_space.groupby(s_tmp.values)['Age'].mean() # 순서상 바꾼게 없어 인덱스를 조작하기 전과 동일하게 나옵니다.

     29.148718
A    35.161943
B    33.622715
C    34.886145
D    33.518201
E    30.068925
F    28.430495
G    24.062775
T    37.000000
Name: Age, dtype: float64

In [13]:
df_space.sample(frac=1.0).groupby(s_tmp.values)['Age'].mean() # 순서르 바꾸면 값이 다르게 나옵니다.

     29.276923
A    29.083665
B    28.906332
C    29.178962
D    27.795745
E    28.393519
F    29.050165
G    28.754785
T    22.600000
Name: Age, dtype: float64

In [14]:
df_space.sample(frac=1.0).groupby(
    df_space['Cabin'].str.split('/').apply(lambda x: x[0] if type(x) == list else '')
)['Age'].mean() # Series로 한다면 중간에 순서가 바뀌더라도 결과는 달라지지 않습니다.

Cabin
     29.148718
A    35.161943
B    33.622715
C    34.886145
D    33.518201
E    30.068925
F    28.430495
G    24.062775
T    37.000000
Name: Age, dtype: float64

### Case 4

groupby - by: function 를 넘겨, PassengerId의 _ 앞자리로 그룹를 기준으로 구하여, VRDeck의 평균을 구합니다.

In [15]:
df_space.set_index('PassengerId').groupby(lambda x: x.split('_')[0])['VRDeck'].mean()

0001       0.000000
0002      44.000000
0003     121.000000
0004       2.000000
0005       0.000000
           ...     
9275     121.333333
9276      74.000000
9278       0.000000
9279       0.000000
9280    1623.500000
Name: VRDeck, Length: 6217, dtype: float64

In [16]:
df_space['PassengerId'].str.extract('(?P<pid_1>\w+)_(?P<pid_2>\w+)')

,pid_1,pid_2
0,0001,01
1,0002,01
2,0003,01
3,0003,02
4,0004,01
...,...,...
8688,9276,01
8689,9278,01
8690,9279,01
8691,9280,01


## by외의 Parameters

pd.DataFrame.groupby(self, by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False)

axis - groupby 연산의 진행 방향

level - 인덱스의 level

as_index - 그룹의 기준 축(거의 쓸 일이 없습니다.)

sort - sort는 그룹의 기준으로 정렬 여부

group_keys - Combine 할 때 그룹값을 첨부 여부

squeeze - Grouping 후에 최대한 낮은 차원으로 반환합니다. (시험장 버젼에서는 동작하지 않습니다)

observed - 그룹 기준에 pd.Categoricals (카테고리형 변수)가 포함 되면, 결과 값에 관측되지 않은 값을 포함 여부


In [17]:
# PassengerId를 _을 기준으로 나누어 앞은 pid_1, 뒤는 pid_2로 합니다.
df_idx = df_space['PassengerId'].str.split('_', expand=True).rename(columns={0: 'pid_1', 1: 'pid_2'})
pd.MultiIndex.from_frame(df_idx) # pid_1, pid_2를 인덱스로 만듭니다.

MultiIndex([('0001', '01'),
            ('0002', '01'),
            ('0003', '01'),
            ('0003', '02'),
            ('0004', '01'),
            ('0005', '01'),
            ('0006', '01'),
            ('0006', '02'),
            ('0007', '01'),
            ('0008', '01'),
            ...
            ('9272', '02'),
            ('9274', '01'),
            ('9275', '01'),
            ('9275', '02'),
            ('9275', '03'),
            ('9276', '01'),
            ('9278', '01'),
            ('9279', '01'),
            ('9280', '01'),
            ('9280', '02')],
           names=['pid_1', 'pid_2'], length=8693)

In [18]:
df_space2 = df_space.set_index(pd.MultiIndex.from_frame(df_idx))
df_space2

PassengerId HomePlanet CryoSleep     Cabin    Destination   Age  \
pid_1 pid_2                                                                   
0001  01        0001_01     Europa     False     B/0/P    TRAPPIST-1e  39.0   
0002  01        0002_01      Earth     False     F/0/S    TRAPPIST-1e  24.0   
0003  01        0003_01     Europa     False     A/0/S    TRAPPIST-1e  58.0   
      02        0003_02     Europa     False     A/0/S    TRAPPIST-1e  33.0   
0004  01        0004_01      Earth     False     F/1/S    TRAPPIST-1e  16.0   
...                 ...        ...       ...       ...            ...   ...   
9276  01        9276_01     Europa     False    A/98/P    55 Cancri e  41.0   
9278  01        9278_01      Earth      True  G/1499/S  PSO J318.5-22  18.0   
9279  01        9279_01      Earth     False  G/1500/S    TRAPPIST-1e  26.0   
9280  01        9280_01     Europa     False   E/608/S    55 Cancri e  32.0   
      02        9280_02     Europa     False   E/608/S    TRAPPIST-1e  44.0   

               VIP  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
pid_1 pid_2                                                                
0001  01     False          0.0        0.0           0.0     0.0     0.0   
0002  01     False        109.0        9.0          25.0   549.0    44.0   
0003  01      True         43.0     3576.0           0.0  6715.0    49.0   
      02     False          0.0     1283.0         371.0  3329.0   193.0   
0004  01     False        303.0       70.0         151.0   565.0     2.0   
...            ...          ...        ...           ...     ...     ...   
9276  01      True          0.0     6819.0           0.0  1643.0    74.0   
9278  01     False          0.0        0.0           0.0     0.0     0.0   
9279  01     False          0.0        0.0        1872.0     1.0     0.0   
9280  01     False          0.0     1049.0           0.0   353.0  3235.0   
      02     False        126.0     4688.0           0.0     0.0    12.0   

                          Name  Transported  
pid_1 pid_2                                  
0001  01       Maham Ofracculy        False  
0002  01          Juanna Vines         True  
0003  01         Altark Susent        False  
      02          Solam Susent        False  
0004  01     Willy Santantines         True  
...                        ...          ...  
9276  01     Gravior Noxnuther        False  
9278  01       Kurta Mondalley        False  
9279  01          Fayey Connon         True  
9280  01      Celeon Hontichre        False  
      02      Propsh Hontichre         True  

[8693 rows x 14 columns]

### Case 1

pid_1 (level=0)로 구분하여 RoomService의 합을 구하세요

In [19]:
df_space2.groupby(level=0)['RoomService'].sum()

pid_1
0001      0.0
0002    109.0
0003     43.0
0004    303.0
0005      0.0
        ...  
9275      1.0
9276      0.0
9278      0.0
9279      0.0
9280    126.0
Name: RoomService, Length: 6217, dtype: float64

In [20]:
# 인덱스에 이름이 있으니, 이름으로 해도 됩니다.
df_space2.groupby('pid_1')['RoomService'].sum()

pid_1
0001      0.0
0002    109.0
0003     43.0
0004    303.0
0005      0.0
        ...  
9275      1.0
9276      0.0
9278      0.0
9279      0.0
9280    126.0
Name: RoomService, Length: 6217, dtype: float64

### Case 2

pid_1 (level=0), pid_2 (level=1) 로 구분하여 RoomService의 합을 구하세요

In [21]:
df_space2.groupby(level=[0, 1])['RoomService'].sum()

pid_1  pid_2
0001   01         0.0
0002   01       109.0
0003   01        43.0
       02         0.0
0004   01       303.0
                ...  
9276   01         0.0
9278   01         0.0
9279   01         0.0
9280   01         0.0
       02       126.0
Name: RoomService, Length: 8693, dtype: float64

In [22]:
# 역시, 인덱스에 이름이 있으니, 이름으로 해도 됩니다.
df_space2.groupby(['pid_1', 'pid_2'])['RoomService'].sum()

pid_1  pid_2
0001   01         0.0
0002   01       109.0
0003   01        43.0
       02         0.0
0004   01       303.0
                ...  
9276   01         0.0
9278   01         0.0
9279   01         0.0
9280   01         0.0
       02       126.0
Name: RoomService, Length: 8693, dtype: float64

In [23]:
# as_index=False: GroupBy에 기준값을 Index에 두지 않고 Column에 둡니다. 
df_space2.groupby('HomePlanet', as_index=False)['RoomService'].sum()

,HomePlanet,RoomService
0,Earth,616094.0
1,Europa,304261.0
2,Mars,952642.0


In [24]:
# HomePlanet 별로 상단 3개씩 가져옵니다.
df_space.groupby('HomePlanet').apply(lambda x: x.head(3))

PassengerId HomePlanet CryoSleep  Cabin    Destination   Age  \
HomePlanet                                                                   
Earth      1      0002_01      Earth     False  F/0/S    TRAPPIST-1e  24.0   
           4      0004_01      Earth     False  F/1/S    TRAPPIST-1e  16.0   
           5      0005_01      Earth     False  F/0/P  PSO J318.5-22  44.0   
Europa     0      0001_01     Europa     False  B/0/P    TRAPPIST-1e  39.0   
           2      0003_01     Europa     False  A/0/S    TRAPPIST-1e  58.0   
           3      0003_02     Europa     False  A/0/S    TRAPPIST-1e  33.0   
Mars       12     0009_01       Mars     False  F/1/P    TRAPPIST-1e  32.0   
           16     0014_01       Mars     False  F/3/P    55 Cancri e  27.0   
           18     0016_01       Mars      True  F/5/P    TRAPPIST-1e  45.0   

                 VIP  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
HomePlanet                                                                   
Earth      1   False        109.0        9.0          25.0   549.0    44.0   
           4   False        303.0       70.0         151.0   565.0     2.0   
           5   False          0.0      483.0           0.0   291.0     0.0   
Europa     0   False          0.0        0.0           0.0     0.0     0.0   
           2    True         43.0     3576.0           0.0  6715.0    49.0   
           3   False          0.0     1283.0         371.0  3329.0   193.0   
Mars       12  False         73.0        0.0        1123.0     0.0   113.0   
           16  False       1286.0      122.0           NaN     0.0     0.0   
           18  False          0.0        0.0           0.0     0.0     0.0   

                            Name  Transported  
HomePlanet                                     
Earth      1        Juanna Vines         True  
           4   Willy Santantines         True  
           5   Sandie Hinetthews         True  
Europa     0     Maham Ofracculy        False  
           2       Altark Susent        False  
           3        Solam Susent        False  
Mars       12       Berers Barne         True  
           16        Flats Eccle        False  
           18         Alus Upead         True

In [25]:
# HomePlanet 별로 상단 3개씩 가져옵니다. 그룹을 키에 포함하지 않습니다.
df_space.groupby('HomePlanet', group_keys=False).apply(lambda x: x.head(3))

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
12,0009_01,Mars,False,F/1/P,TRAPPIST-1e,32.0,False,73.0,0.0,1123.0,0.0,113.0,Berers Barne,True
16,0014_01,Mars,False,F/3/P,55 Cancri e,27.0,False,1286.0,122.0,NaN,0.0,0.0,Flats Eccle,False
18,0016_01,Mars,True,F/5/P,TRAPPIST-1e,45.0,False,0.0,0.0,0.0,0.0,0.0,Alus Upead,True


# Group by의 주요 연산

그룹으로 나누어 데이터 처리를 합니다.

pandas를 통한 Group-By 연산은 크게 apply, aggregation, transform, filter 로 나눕니다. 
    
aggregation(agg): 분할(Split) → 집계(Aggregation) → 결합(Combine)
       
      Series 별로 하나의 통계값으로 변환. 
      
      agg에 함수를 전달하면 각각의 Series 값이 전달된다.
      
      .agg(lambda x: x...)  
      
      Ex) 그룹별로 평균을 구하라.
    
transform(transform): 분할(Split) → 변환(Transform) → 결합(Combine)
     
      Series의 행에 대응하는 Series를 반환합니다.
     
      Ex) 그룹별로 표준화를 하라
          그룹별 평균으로 결측값을 채워라

filtering(filter): Split → 불리언 집계(Aggregation into boolean variable) → 결합(Combine)

      그룹별로 boolean 결과를 반환하여 True인  결과를 내는 그룹만을 결합한다.
      
      Ex) 그룹에 해당하는 수가 10개 이상인 관측값만을 가져온다.

apply(apply): Split → Process (Aggregation, Transform, 포함 ) → 결합(Combine)
       
     여러 컬럼을 대상으로 apply를 한다면, apply에 callback 함수에 DataFrame가 넘어옵니다. 
    
     aggregation, transform 연산을 apply로도 구현할 수 있습니다.
     
     
pd.DataFrame.groupby는 DataFrameGroupBy, pd.Series.groupby는 SeriesGroupBy 를 반환합니다.

이를 통한 연산처리 과정을 설명합니다.

In [26]:
# 모든 데이터프레임에 대해서는 pandas.core.groupby.generic.DataFrameGroupBy
df_space2.groupby('HomePlanet')

In [27]:
# Series도 가능합니다. pandas.core.groupby.generic.SeriesGroupBy
df_space2['RoomService'].groupby(df_space2['HomePlanet'])

In [28]:
# 컬럼명을 지칭할 경우 pandas.core.groupby.generic.SeriesGroupBy
df_space2.groupby('HomePlanet')['RoomService']

In [29]:
# 리스트일 경우 pandas.core.groupby.generic.DataFrameGroupBy
df_space2.groupby('HomePlanet')[['RoomService']]

## Aggregation

**pd.core.groupby.DataFrameGroupBy.agg**

pd.core.groupby.DataFrameGroupBy.agg(self, func, \*args, \*\*kwargs)

func: function, str, list or dict

    str - 집계 함수 이름 (sum, mean, std, var, count) ※ std, var는 표본표준편차, 표본분산 입니다.
     
    list - 집계 함수 이름, 함수
      
    dict - {변수명: function 또는 list} : 변수별 집계 연산 정의
      
    --------------------------------------------------------
    function의 구조 
      
    def aggfunc(x, *args):
      
       x: Series - groupby에 의해 분리된 DataFrame 또는 Series의 각각의 변수들이 Series로 들어옵니다.

**kwargs:

    결과컬럼명=pd.NamedAgg 객체
    
    pd.NamedAgg에 정의된 집계 연산 결과를 정의한 '결과컬럼명'으로 컬럼를 생성하여 반환합니다.
    
pd.NamedAgg(대상변수명, function or str)

### Case 1

HomePlanet 별 ShoppingMall의 합을 ShppingMall_sum를 변수명으로 하여 집계하세요.

In [30]:
# DataFrameGroupBy에서 하나의 컬럼을 선택했으므로(['ShoppingMall']) SeriesGroupBy가 됩니다.
# 내부에서 사용하는 Data구조가 Series 일 뿐이지 거의 같습니다.
df_space.groupby('HomePlanet')['ShoppingMall'].agg('sum').rename('ShppingMall_sum') # rename을 하지 않으면 ShoppingMall로 생성됩니다.

HomePlanet
Earth     601088.0
Europa    314054.0
Mars      531452.0
Name: ShppingMall_sum, dtype: float64

### Case 2

HomePlanet 별 ShoppingMall의 평균과 분산을 ShoppingMall_mean, ShoppingMall_var를 변수명으로 하여 집계하세요.

In [31]:
# 복수의 연산은 list로 합니다.

df_space.groupby('HomePlanet')['ShoppingMall'].agg(['mean', 'var'])\
        .rename(columns={'mean': 'ShoppingMall_mean', 'var': 'ShoppingMall_var'})

,ShoppingMall_mean,ShoppingMall_var
HomePlanet,,
Earth,133.872606,120073.341753
Europa,151.132820,828030.757507
Mars,308.445734,429276.451612


In [32]:
df_space.groupby('HomePlanet')['ShoppingMall'].agg(
    ShoppingMall_mean='mean',
    ShoppingMall_var='var'
)

,ShoppingMall_mean,ShoppingMall_var
HomePlanet,,
Earth,133.872606,120073.341753
Europa,151.132820,828030.757507
Mars,308.445734,429276.451612


### Case 3

HomePlanet 별 log(Shopping + 1)의  합을 ShoppingMall_log_sum로 집계하세요

In [33]:
# 여러 방법 중 function을 이용하여 집계를 해봅니다.
df_space.groupby('HomePlanet')['ShoppingMall'].agg(
    ShoppingMall_log_sum = lambda x: np.sum(np.log1p(x))
)

,ShoppingMall_log_sum
HomePlanet,
Earth,7438.770053
Europa,1673.170317
Mars,4505.007400


In [34]:
# 참고: 다른 방법, 아래와 같이 여러 연산을 할 경우에 유리합니다.
df_space[['HomePlanet', 'ShoppingMall']].assign( # assign읜 Deep-Copy를 하므로 연산에 필요한 컬럼만 선택합니다.
    ShoppingMall_log=lambda x: np.log1p(x['ShoppingMall'])
).groupby('HomePlanet')['ShoppingMall_log'].agg(
    ShoppingMall_log_sum='sum',
    ShoppingMall_log_cnt='mean',
    ShoppingMall_log_std='std'
)

,ShoppingMall_log_sum,ShoppingMall_log_cnt,ShoppingMall_log_std
HomePlanet,,,
Earth,7438.770053,1.656742,2.530121
Europa,1673.170317,0.805183,2.014774
Mars,4505.007400,2.614630,2.984148


### Case 4

HomePlanet 별 RoomService, FoodCourt, ShoppingMall의 평균을 RoomSerivce_mean, FoodCourt_mean, ShoppingMall_mean을 변수명으로 집계하세요

In [35]:
df_space.groupby('HomePlanet')[['RoomService', 'FoodCourt', 'ShoppingMall']].agg('mean')\
        .rename(columns=lambda x: x + '_mean')

,RoomService_mean,FoodCourt_mean,ShoppingMall_mean
HomePlanet,,,
Earth,136.940209,137.228857,133.872606
Europa,145.231981,1503.212950,151.132820
Mars,552.897272,54.361999,308.445734


In [36]:
# 다른 방법
# DataFrameGroupBy 로 **kargs를 이용하여 명명하면서 처리하려면, 대상변수를 지정이 필요하므로, 이를 위한 NamedAgg를 사용합니다.
df_space.groupby('HomePlanet')[['RoomService', 'FoodCourt', 'ShoppingMall']].agg(
    RoomService_mean=pd.NamedAgg('RoomService', 'mean'),
    FoodCourt_mean=pd.NamedAgg('FoodCourt', 'mean'),
    ShoppingMall_mean=pd.NamedAgg('ShoppingMall', 'mean'),
)

,RoomService_mean,FoodCourt_mean,ShoppingMall_mean
HomePlanet,,,
Earth,136.940209,137.228857,133.872606
Europa,145.231981,1503.212950,151.132820
Mars,552.897272,54.361999,308.445734


In [37]:
# 위 과정을 Dictionary Comprehension과 Unpacking을 이용하여 간단히 해봅니다.
cols = ['RoomService', 'FoodCourt', 'ShoppingMall']
df_space.groupby('HomePlanet')[cols].agg(
    **{i + '_mean': pd.NamedAgg(i, 'mean') for i in cols}
)

,RoomService_mean,FoodCourt_mean,ShoppingMall_mean
HomePlanet,,,
Earth,136.940209,137.228857,133.872606
Europa,145.231981,1503.212950,151.132820
Mars,552.897272,54.361999,308.445734


### Case 5: 

HomePlanet 별 RoomService, FoodCourt, ShoppingMall의 평균과 표준편차를 

각각의 평균은 RoomSerivce_mean, FoodCourt_mean, ShoppingMall_mean으로 
 
각각의 표준편차는 RoomSerivce_std, FoodCourt_std, ShoppingMall_std로 
 
변수명으로 집계한 데이터프레임을 만드세요

In [38]:
df_tmp = df_space.groupby('HomePlanet')[['RoomService', 'FoodCourt', 'ShoppingMall']].agg(['mean', 'std'])
df_tmp.columns = df_tmp.columns.map(lambda x: x[0] + '_' + x[1])
df_tmp

,RoomService_mean,RoomService_std,FoodCourt_mean,FoodCourt_std,ShoppingMall_mean,ShoppingMall_std
HomePlanet,,,,,,
Earth,136.940209,381.408218,137.228857,368.327870,133.872606,346.516005
Europa,145.231981,791.140422,1503.212950,2956.746517,151.132820,909.961954
Mars,552.897272,947.611891,54.361999,283.157183,308.445734,655.191920


In [39]:
# python 내장 모듈 itertools의 product 이용
import itertools

# itertools.product 는 입력된 리스트의 조합에 대한 iterable을 제공합니다
for i, j in itertools.product(['RoomService', 'FoodCourt', 'ShoppingMall'], ['mean', 'std']):
    print(i, j)

RoomService mean
RoomService std
FoodCourt mean
FoodCourt std
ShoppingMall mean
ShoppingMall std


In [40]:
# 유사 함수로. pandas.MultiIndex.from_product 가 있습니다. 반환은 pd.MultiIndex 객체가 됩니다.
for i, j in pd.MultiIndex.from_product([['RoomService', 'FoodCourt', 'ShoppingMall'], ['mean', 'std']]):
    print(i, j)

RoomService mean
RoomService std
FoodCourt mean
FoodCourt std
ShoppingMall mean
ShoppingMall std


In [41]:
# itertools.product와 dictionary comprehension, unpacking을 이용
cols, aggs = ['RoomService', 'FoodCourt', 'ShoppingMall'], ['mean', 'std']
df_space.groupby('HomePlanet')[cols].agg(
    **{i + '_' + j: pd.NamedAgg(i, j) for i, j in itertools.product(cols, aggs)}
)

,RoomService_mean,RoomService_std,FoodCourt_mean,FoodCourt_std,ShoppingMall_mean,ShoppingMall_std
HomePlanet,,,,,,
Earth,136.940209,381.408218,137.228857,368.327870,133.872606,346.516005
Europa,145.231981,791.140422,1503.212950,2956.746517,151.132820,909.961954
Mars,552.897272,947.611891,54.361999,283.157183,308.445734,655.191920


※ SeriesGroupBy의 agg는 function에 groupby에 의해 구분된 Series를 넘겨 주고,

   DataFrameGroupBy의 agg는 function에 groupby에 의해 구분된 Series를 컬럼별로 넘겨줍니다.

In [42]:
def examine_agg(x):
    print(type(x), x.name, x.index.max())
    return x.name

df_space.groupby('HomePlanet')[['RoomService', 'FoodCourt', 'ShoppingMall']].agg(
    lambda x: examine_agg(x)
)

<class 'pandas.core.series.Series'> RoomService 8690
<class 'pandas.core.series.Series'> RoomService 8692
<class 'pandas.core.series.Series'> RoomService 8668
<class 'pandas.core.series.Series'> FoodCourt 8690
<class 'pandas.core.series.Series'> FoodCourt 8692
<class 'pandas.core.series.Series'> FoodCourt 8668
<class 'pandas.core.series.Series'> ShoppingMall 8690
<class 'pandas.core.series.Series'> ShoppingMall 8692
<class 'pandas.core.series.Series'> ShoppingMall 8668


,RoomService,FoodCourt,ShoppingMall
HomePlanet,,,
Earth,RoomService,FoodCourt,ShoppingMall
Europa,RoomService,FoodCourt,ShoppingMall
Mars,RoomService,FoodCourt,ShoppingMall


### Case 6

HomePlanet별로 'RoomService', 'FoodCourt', 'ShoppingMall'에 변동계수
    
    CV(변동계수)=$\frac{s}{\bar{x}}$
    
를 구하여 컬럼며이 RoomService_cv, FoodCourt_cv, ShoppingMall_cv 인 데이터프레임을 만드세요

In [43]:
def cv(x):
    return x.std() / x.mean()

cols = ['RoomService', 'FoodCourt', 'ShoppingMall']
df_space.groupby('HomePlanet')[cols].agg(cv).rename(columns={i: i + '_cv' for i in cols})

,RoomService_cv,FoodCourt_cv,ShoppingMall_cv
HomePlanet,,,
Earth,2.785217,2.684041,2.588401
Europa,5.447426,1.966951,6.020942
Mars,1.713902,5.208734,2.124172


In [44]:
# lambda로 표현 가능하면 lambda로도 씁니다.
df_space.groupby('HomePlanet')[cols].agg(lambda x: x.std() / x.mean())\
        .rename(columns={i: i + '_cv' for i in cols})

,RoomService_cv,FoodCourt_cv,ShoppingMall_cv
HomePlanet,,,
Earth,2.785217,2.684041,2.588401
Europa,5.447426,1.966951,6.020942
Mars,1.713902,5.208734,2.124172


### Case 7

RoomSerivce에 결측인 PassengerId의 HomePlanet 별로 list를 만드세요

In [45]:
# list도 함수이므로 agg에 list를 넣어주면 HomePlanet 별로 PassengerId의 list 가 만들어지게 됩니다.
df_space.loc[df_space['RoomService'].isna()].groupby('HomePlanet')['PassengerId'].agg(list)

HomePlanet
Earth     [0020_05, 0091_01, 0234_01, 0250_01, 0357_01, ...
Europa    [0310_01, 0438_01, 0439_01, 1294_03, 1907_01, ...
Mars      [0031_03, 0141_01, 0193_02, 0476_01, 1100_02, ...
Name: PassengerId, dtype: object

### Case 8

HomePlanet로 구분했을 때, Age가 등분산성을 보이는지 검정하세요. 결측값은 제외합니다.

      등분산성 검정 방법은 Bartlett을 사용합니다.
      
      [Hint] scipy.stats.bartlett

In [46]:
df_space.groupby('HomePlanet')['Age'].agg(lambda x: list(x.loc[x.notna()]))

HomePlanet
Earth     [24.0, 16.0, 44.0, 26.0, 28.0, 35.0, 48.0, 28....
Europa    [39.0, 58.0, 33.0, 14.0, 34.0, 45.0, 62.0, 34....
Mars      [32.0, 27.0, 45.0, 21.0, 43.0, 47.0, 2.0, 20.0...
Name: Age, dtype: object

In [47]:
from scipy.stats import bartlett

# Series를 list로 변환하고, unpacking을 통해 bartlett에 전달합니다.
bartlett(*df_space.groupby('HomePlanet')['Age'].agg(lambda x: list(x.loc[x.notna()])).tolist())

BartlettResult(statistic=27.601244720365813, pvalue=1.0149995790670188e-06)

## Transform

구분된 데이터(DataFrame, Series)와 1:1로 변환된 데이터를 넘겨 줍니다.

입력된 Series 또는 DataFrame과 동일한 Index로 반환됩니다.

**pd.core.groupby.DataFrameGroupBy.transform(self, func)**

func: function

    1:1 변환함수
    
    N:1 변환함수, 즉 집계 함수면 그룹별로 집계된 값으로 동일한 모양으로 값을 반환합니다.  (예제 참조)

In [48]:
def exam_transform_1(x):
    """
        transform에 동일 형태의 Series를 반환했을 때를 살펴보기 위한 함수입니다.
    """
    print(type(x), x.name, x.shape)
    if type(x) == pd.DataFrame: # 첫 번째 그룹을 마무리 할 때 첫번째 작업한 데이터프레임을 넘겨준다. 
        raise "AAA" # 작업하는 DataFrame의 구성을 확인하기 위해 제공하는 듯합니다. 에러가 발생해도 무관합니다.
    return (x - x.mean()) / x.std()

In [49]:
def exam_transform_2(x):
    """
        transform에 단일값을 반환했을 때를 살펴보기 위한 함수입니다.
    """
    print(type(x), x.name, x.shape)
    if type(x) == pd.DataFrame: # 첫 번째 그룹을 마무리 할 때 첫번째 작업한 데이터프레임을 넘겨준다. 
        raise "AAA" # 작업하는 DataFrame의 구성을 확인하기 위해 제공하는 듯합니다. 에러가 발생해도 무관합니다.
    return x.mean()

In [50]:
# 다른 GroupBy는 그룹의 기준 컬럼에 결측이 있으면 무시했지만
# 시험장 버젼의 pandas에서는 이로 인해 오류가 발생합니다.
# 일관적으로 무시하지 않는 것으로 보아 이후 버젼에는 보완이 됐으리라 생각됩니다.
df_space.groupby('HomePlanet')[['RoomService', 'ShoppingMall']].transform(exam_transform_1)

<class 'pandas.core.series.Series'> RoomService (4602,)
<class 'pandas.core.series.Series'> RoomService (4602,)
<class 'pandas.core.series.Series'> ShoppingMall (4602,)
<class 'pandas.core.frame.DataFrame'> Earth (4602, 2)
<class 'pandas.core.series.Series'> RoomService (2131,)
<class 'pandas.core.series.Series'> RoomService (2131,)
<class 'pandas.core.series.Series'> ShoppingMall (2131,)
<class 'pandas.core.series.Series'> RoomService (1759,)
<class 'pandas.core.series.Series'> RoomService (1759,)
<class 'pandas.core.series.Series'> ShoppingMall (1759,)


ValueError: Length mismatch: Expected axis has 8492 elements, new values have 8693 elements

In [51]:
df_space['HomePlanet'].isna().sum()

201

In [52]:
df_space2 = df_space.assign(
    HomePlanet=lambda x: x['HomePlanet'].fillna('')
) # HomePlanet의 결측을 ''로 채우고 따로 df_space2로 만듭니다.
df_space2['HomePlanet'].isna().sum()

0

In [53]:
# Series를 반환할 경우 첫 번째 Series를 다시 한번 호출합니다.
df_space2.groupby('HomePlanet')[['RoomService', 'ShoppingMall']].transform(exam_transform_1)

<class 'pandas.core.series.Series'> RoomService (201,)
<class 'pandas.core.series.Series'> RoomService (201,)
<class 'pandas.core.series.Series'> ShoppingMall (201,)
<class 'pandas.core.frame.DataFrame'>  (201, 2)
<class 'pandas.core.series.Series'> RoomService (4602,)
<class 'pandas.core.series.Series'> RoomService (4602,)
<class 'pandas.core.series.Series'> ShoppingMall (4602,)
<class 'pandas.core.series.Series'> RoomService (2131,)
<class 'pandas.core.series.Series'> RoomService (2131,)
<class 'pandas.core.series.Series'> ShoppingMall (2131,)
<class 'pandas.core.series.Series'> RoomService (1759,)
<class 'pandas.core.series.Series'> RoomService (1759,)
<class 'pandas.core.series.Series'> ShoppingMall (1759,)


,RoomService,ShoppingMall
0,-0.183573,-0.166087
1,-0.073255,-0.314192
2,-0.129221,-0.166087
3,-0.183573,0.241622
4,0.435386,0.049427
...,...,...
8688,-0.183573,-0.166087
8689,-0.359038,-0.386339
8690,-0.359038,5.016009
8691,-0.183573,-0.166087


In [54]:
# 단일값을 호출할 경우에는 첫번째 호출을 다시 하지는 않습니다.
df_space2.groupby('HomePlanet')[['RoomService', 'ShoppingMall']].transform(exam_transform_2)

<class 'pandas.core.series.Series'> RoomService (201,)
<class 'pandas.core.series.Series'> ShoppingMall (201,)
<class 'pandas.core.frame.DataFrame'>  (201, 2)
<class 'pandas.core.series.Series'> RoomService (4602,)
<class 'pandas.core.series.Series'> ShoppingMall (4602,)
<class 'pandas.core.series.Series'> RoomService (2131,)
<class 'pandas.core.series.Series'> ShoppingMall (2131,)
<class 'pandas.core.series.Series'> RoomService (1759,)
<class 'pandas.core.series.Series'> ShoppingMall (1759,)


,RoomService,ShoppingMall
0,145.231981,151.132820
1,136.940209,133.872606
2,145.231981,151.132820
3,145.231981,151.132820
4,136.940209,133.872606
...,...,...
8688,145.231981,151.132820
8689,136.940209,133.872606
8690,136.940209,133.872606
8691,145.231981,151.132820


※ 확인한 바와 같이 transform은 내부 호출 구조에서는 Series와 DataFrame를 넘기기도 합니다.

이에 대응하여 코딩하는 것은 무의미해 보이며, 

transform이 넘겨주는 값이 Series인 경우만을 고려하여 코드를 작성하면 됩니다.

차라리, apply를 사용하는 것이 보다 일반적일 수도 있겠지만 시험장 버젼의 apply는 결합후 인덱스 처리할 때 번거로움이 있습니다. 

apply의 인덱스 처리시 번거로움이 없고, 단순 1:1 변환에 사용한다는 관점으로 접근하면 됩니다.

### Case 1

df_space2에서 HomePlanet 별로 RoomService를 표준화 변환합니다.

In [55]:
df_space2.groupby('HomePlanet')['RoomService'].transform(lambda x: (x - x.mean()) / x.std())

0      -0.183573
1      -0.073255
2      -0.129221
3      -0.183573
4       0.435386
          ...   
8688   -0.183573
8689   -0.359038
8690   -0.359038
8691   -0.183573
8692   -0.024309
Name: RoomService, Length: 8693, dtype: float64

이 Series를 RoomService_std란 이름으로 추가 하세요.

In [56]:
# 입력과 동일한 인덱스를 가지고 나오므로 바로 대입해주면 됩니다.
df_space2['RoomService_std'] = df_space2.groupby('HomePlanet')['RoomService'].transform(lambda x: (x - x.mean()) / x.std())
df_space2.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,RoomService_std
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,-0.183573
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,-0.073255
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,-0.129221
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,-0.183573
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0.435386


### Case 2

df_space2의 RoomService, ShoppingMall 에는 결측치가 존재합니다. 결측치를 HomePlanet별 RoomService와 ShoppingMall 평균으로 대체해서, RoomSerivce_fn, ShoppingMall_fn 컬럼으로 추가합니다.

In [57]:
df_space2[['RoomService_fn', 'ShoppingMall_fn']] = \
        df_space2.groupby('HomePlanet')[['RoomService', 'ShoppingMall']]\
                .transform(
                    lambda x: x.fillna(x.mean()) # 각각이 Series로 넘어온다고 생각하고 결측을 대체하면 됩니다.
                )

### Case 3

df_space2에서 HomePlanet 별로 RoomService의 순위(Rank)를 구하여 RoomService_rnk 컬럼을 추가합니다.

※ RoomService가 클수록 높은 순위(숫자상으로 낮은)이고, 동점일 경우에는 가장 낮은 순위(숫자상으로 높은)를 동일하게 취합니다. 결측값일 경우에는 가장 낮은 순위로 합니다. 순위는 가장 높은 순위는 1이며, 자연수입니다.

In [58]:
df_space2['RoomService_rnk'] = df_space2.groupby('HomePlanet')['RoomService'].transform(
    lambda x: x.rank(method='max', ascending=False, na_option='bottom')
)

In [59]:
# 결과를 확인합니다.
df_space2.loc[df_space['HomePlanet'] == 'Europa', ['RoomService', 'RoomService_rnk']]\
        .sort_values('RoomService', ascending=False).iloc[20:30]

,RoomService,RoomService_rnk
1666,3677.0,21.0
5945,3616.0,22.0
707,3594.0,23.0
7055,3580.0,24.0
496,3573.0,25.0
2493,3551.0,26.0
3677,3478.0,28.0
5808,3478.0,28.0
3467,3340.0,29.0
5600,3215.0,30.0


In [60]:
# Transform을 쓰지 않아도 SeriesGroupBy의 메소드를 써도 됩니다.
df_space2.groupby('HomePlanet')['RoomService'].rank(method='max', ascending=False, na_option='bottom')

0       2095.0
1        893.0
2        209.0
3       2095.0
4        660.0
         ...  
8688    2095.0
8689    4499.0
8690    4499.0
8691    2095.0
8692     162.0
Name: RoomService, Length: 8693, dtype: float64

## Apply

구분된 데이터를 DataFrameGroupBy는 DataFrame으로 SeriesGroupBy는 Series로 넘겨 줍니다.

Apply를 통해 반환할 수 있는 데이터 구조는 단일값 또는 단일객체 / Series / DataFrame 입니다. 

시험장 버젼에서는, 

입력된 데이터와 동일한 Index를 가지는 Series / DataFrame이 반환될 경우에는

결합되어 반환된 데이터프레임의 인덱스에 그룹값이 포함되지 않습니다. 

이 이외의 경우에는 

최근 버젼에서는 동일한 행의 수를 지닌 numpy.ndarray에 대해서도 그룹값이 생략할 수 있습니다.

**pd.core.groupby.DataFrameGroupBy.apply(self, func)**

func: callable

       DataFrame을 입력으로 받고 단일값 또는 단일객체 / Series / DataFrame / numpy.ndarray
       
       ※ DataFrame의 name attribute는 aggregation / transform과 달리 그룹값이 넘어옵니다.
       
          이는 넓은 활용성을 부여합니다.

In [61]:
def examine_apply_1(x):
    """
    단일값을 반환할 때
    """
    print(type(x), x.name, x.shape)
    return len(x)

def examine_apply_2(x):
    """
    Series로 여러 값을 반환할 때
    """
    print(type(x), x.name, x.shape)
    if type(x) == pd.Series:
        return pd.Series([len(x), x.notna().sum()], index=['len', 'notna'])
    else:
        return pd.Series([len(x), x.notna().sum().sum()], index=['len', 'notna'])

def examine_apply_3(x):
    """
    동일한 Index로 반환할 때
    """
    print(type(x), x.name, x.shape)
    return x + 1

def examine_apply_3_2(x):
    """
    인덱스를 새로 정의하여 반환할 때, Index로 반환할 때
    """
    print(type(x), x.name, x.shape)
    return (x + 1).reset_index()

def examine_apply_3_3(x):
    """
    인덱스는 같지만 순서가 다를 때
    """
    print(type(x), x.name, x.shape)
    return (x + 1).sample(frac=1)

In [62]:
# SeriesGroupBy를 넘길 경우, 단일값 반환합니다.
# 언급한대로 name에  컬럼명이 들어가는게 아니라 그룹값이 들어갑니다.
df_space2.groupby('HomePlanet')['RoomService'].apply(
    examine_apply_1
)

<class 'pandas.core.series.Series'>  (201,)
<class 'pandas.core.series.Series'> Earth (4602,)
<class 'pandas.core.series.Series'> Europa (2131,)
<class 'pandas.core.series.Series'> Mars (1759,)


HomePlanet
           201
Earth     4602
Europa    2131
Mars      1759
Name: RoomService, dtype: int64

In [63]:
# DataFrameGroupBy를 넘기고 단일값 반환하는 경우
df_space2.groupby('HomePlanet')[['RoomService']].apply(
    examine_apply_1
)

<class 'pandas.core.frame.DataFrame'>  (201, 1)
<class 'pandas.core.frame.DataFrame'> Earth (4602, 1)
<class 'pandas.core.frame.DataFrame'> Europa (2131, 1)
<class 'pandas.core.frame.DataFrame'> Mars (1759, 1)


HomePlanet
           201
Earth     4602
Europa    2131
Mars      1759
dtype: int64

In [64]:
# SeriesGroupBy를 넘기고 Series를 반환하는 경우
df_space2.groupby('HomePlanet')['RoomService'].apply(
    examine_apply_2
) # Series로 값을 넘기고 넘어올 때의 Index까지 포함한 Index가 생긴다.

<class 'pandas.core.series.Series'>  (201,)
<class 'pandas.core.series.Series'> Earth (4602,)
<class 'pandas.core.series.Series'> Europa (2131,)
<class 'pandas.core.series.Series'> Mars (1759,)


HomePlanet       
            len       201
            notna     195
Earth       len      4602
            notna    4499
Europa      len      2131
            notna    2095
Mars        len      1759
            notna    1723
Name: RoomService, dtype: int64

In [65]:
# DataFrameGroupBy를 넘기고 Series를 반환하는 경우
df_space2.groupby('HomePlanet')[['RoomService']].apply(
    examine_apply_2
) # pd.Series로 반환할 경우 Series의 index는 컬럼에 배치

<class 'pandas.core.frame.DataFrame'>  (201, 1)
<class 'pandas.core.frame.DataFrame'> Earth (4602, 1)
<class 'pandas.core.frame.DataFrame'> Europa (2131, 1)
<class 'pandas.core.frame.DataFrame'> Mars (1759, 1)


,len,notna
HomePlanet,,
,201,195
Earth,4602,4499
Europa,2131,2095
Mars,1759,1723


In [66]:
# 동일한 인덱스와 순서로 넘어오면 그룹값이 인덱스에 추가되지 않습니다.
df_space2.groupby('HomePlanet')[['RoomService']].apply(
    examine_apply_3
) # pd.Series로 반환할 경우 Series의 index는 컬럼에 배치

<class 'pandas.core.frame.DataFrame'>  (201, 1)
<class 'pandas.core.frame.DataFrame'> Earth (4602, 1)
<class 'pandas.core.frame.DataFrame'> Europa (2131, 1)
<class 'pandas.core.frame.DataFrame'> Mars (1759, 1)


,RoomService
0,1.0
1,110.0
2,44.0
3,1.0
4,304.0
...,...
8688,1.0
8689,1.0
8690,1.0
8691,1.0


In [67]:
# 다른 인덱스로 넘어오면 그룹값이 인덱스에 추가됩니다.
df_space2.groupby('HomePlanet')[['RoomService']].apply(
    examine_apply_3_2
) # Hom

<class 'pandas.core.frame.DataFrame'>  (201, 1)
<class 'pandas.core.frame.DataFrame'> Earth (4602, 1)
<class 'pandas.core.frame.DataFrame'> Europa (2131, 1)
<class 'pandas.core.frame.DataFrame'> Mars (1759, 1)


index  RoomService
HomePlanet                         
           0        59          1.0
           1       113          1.0
           2       186          1.0
           3       225        314.0
           4       234          1.0
...                ...          ...
Mars       1754   8654          1.0
           1755   8655          1.0
           1756   8660          3.0
           1757   8661        700.0
           1758   8668          1.0

[8693 rows x 2 columns]

In [68]:
# 인덱스의 순서가 바뀌어도 그룹값이 붙습니다.
df_space2.groupby('HomePlanet')[['RoomService']].apply(
    examine_apply_3_3
)

<class 'pandas.core.frame.DataFrame'>  (201, 1)
<class 'pandas.core.frame.DataFrame'> Earth (4602, 1)
<class 'pandas.core.frame.DataFrame'> Europa (2131, 1)
<class 'pandas.core.frame.DataFrame'> Mars (1759, 1)


RoomService
HomePlanet                  
           5055          3.0
           6913          1.0
           8319          1.0
           6609          1.0
           3091        667.0
...                      ...
Mars       6840        742.0
           5069        139.0
           3286        810.0
           2350        738.0
           2632          1.0

[8693 rows x 1 columns]

apply에서도 df_space2를 사용합니다.

### Case 1: 

HomePlanet 별로 Age에 대한 정규성 검정하고 검정 통계량은 stats, pvalue는 pval 컬럼으로된 데이터프레임을 만드세요. Age 결측치는 제외합니다.
        정규성 검정은 Jarque-Bera를 사용합니다.
        
    [Hint] scipy.stats.jarque_bera를 사용합니다.

In [69]:
# HomePlanet으로 구분하지 않고 해봅니다.
from scipy.stats import jarque_bera

result = jarque_bera(df_space2.loc[df_space['Age'].notna(), 'Age'])
result

Jarque_beraResult(statistic=252.7789879511956, pvalue=0.0)

In [70]:
# scipy의 Result 시리즈 객체는 이름과 array index로 가져올수 있습니다.
result[0], result[1], result.statistic, result.pvalue

(252.7789879511956, 0.0, 252.7789879511956, 0.0)

In [71]:
# tuple 행태입니다. jarque_bera
df_space2.groupby('HomePlanet')['Age'].apply(
    lambda x: pd.Series(jarque_bera(x.loc[x.notna()]),index=['stats', 'pval'])
).unstack() # 우측 인덱스를 컬럼으로 전환합니다.

,stats,pval
HomePlanet,,
,14.588428,6.794586e-04
Earth,289.463962,0.000000e+00
Europa,54.248068,1.660228e-12
Mars,28.613077,6.119969e-07


### Case 2

HomePlanet별로 Destination의 다음과 같은 형태로 빈도수 테이블을 구하세요. 행과 인덱스의 순서는 일치하지 않아도 됩니다.
    
|  |   |TRAPPIST-1e|PSO J318.5-22|55 Cancri e|
|---|----|----|----|----|
| |OO|OO|OO|OO|OO|
|Earth|OO|OO|OO|OO|OO|
|Europa|OO|OO|OO|OO|OO|
|Mars|OO|OO|OO|OO|OO|

빈 칸은 결측을 의미  합니다

In [72]:
# 여러가지 방법이 보이지만, groupby ~ apply를 써봅니다.
# 이후 여러 방법을 소개합니다.

df_space2_tmp =df_space2[['HomePlanet', 'Destination']].assign(
    Destination=lambda x: x['Destination'].fillna('')
) # 여러 예를 만들 것이므로 변수로 만듭니다.

# x는 Series 이므로 value_counts() 를 사용할 수 있습니다. (시험장 버젼은 Series밖에 value_counts가 안 됩니다)
# value_counts를 하면 Index에는 Series의 값과 내용에는 출현 빈도인 Series를 가져옵니다.
df_space2_tmp.groupby('HomePlanet')['Destination'].apply(lambda x: x.value_counts())\
    .unstack() # 마지막 레벨의 인덱스를 컬럼으로 보냅니다.

,,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
HomePlanet,,,,
,4,31,16,150
Earth,99,690,712,3101
Europa,37,886,19,1189
Mars,42,193,49,1475


In [73]:
# 두 번째 방법. pivot_table

df_space2_tmp.pivot_table(index='HomePlanet', columns='Destination', aggfunc='size')

Destination,,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
HomePlanet,,,,
,4,31,16,150
Earth,99,690,712,3101
Europa,37,886,19,1189
Mars,42,193,49,1475


In [74]:
# 세 번째 방법, crosstab

pd.crosstab(index=df_space2_tmp['HomePlanet'], columns=df_space2_tmp['Destination'])

Destination,,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
HomePlanet,,,,
,4,31,16,150
Earth,99,690,712,3101
Europa,37,886,19,1189
Mars,42,193,49,1475


In [75]:
# 네 번째 방법 groupby를 ['HomePlanet', 'Destination']

df_space2_tmp.groupby(['HomePlanet', 'Destination']).size().unstack()

Destination,,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
HomePlanet,,,,
,4,31,16,150
Earth,99,690,712,3101
Europa,37,886,19,1189
Mars,42,193,49,1475


### Case 3: apply는 이 한 문제로 마무리하겠습니다.
    
    HomePlanet 별로 아래를 입력 변수로 하여 Transported 여부를 맞추는 Logistic Regression 모델 만드세요.

    'CryoSleep', 'Destination', 'Age', 'VIP'
    
    위 변수 중에 하나라도 결측이 있는 행은 제외하여 space3 데이터셋을 만듭니다.

    **전처리 요건**
    
    CrypSleep, Destination, VIP는 범주형 변수이고 알파벳순으로 첫번째 수준은 제외하고 가변수화 합니다.
    
    Age는 HomePlanet 별로 표준화 합니다.
    
    **검증 요건**
    
    행번호가 4의 배수가 아니면 train 셋
    
    행번호가 4의 배수가 이면 test셋 으로 합니다.
    
    test셋 대한 전체에 대한  ROC AUC(Area Under the Receiver Operating Characteristic Curve)와 
    
    HomePlanet 각각에 대한 ROC AUC를 구합니다.
    
    **모델 설정**
    
    sklearn.linear_model.LogisticRegression 
    
    C = 1, random_state=123, solver='lbfgs' 를 사용합니다.
    
    입력변수의 순서는 'CryoSleep', 'Destination', 'Age', 'VIP'로 합니다.

In [76]:
cols = ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'Transported']

df_space3 = df_space2.loc[df_space2[cols].notna().all(axis=1), cols].copy()
df_space3.isna().sum(), df_space3.shape

(HomePlanet     0
 CryoSleep      0
 Destination    0
 Age            0
 VIP            0
 Transported    0
 dtype: int64,
 (7937, 6))

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

def build_model():
    """
    전처리 단계를 포함한 LogisticRegression 파이프라인 모델을 만듭니다.
    이 부분이 이해가 가지 않을 경우에, sklearn_pipeline notebook을 참조해 보세요.
    """
    # ColumnTransformer를 사용하여 범주형과 수치형을 동시에 처리하는 전처리기를 만듭니다.
    ct = ColumnTransformer([
        ('ohe', OneHotEncoder(drop='first'), ['CryoSleep', 'Destination', 'VIP']),
        ('std', StandardScaler(), ['Age'])
    ])
    return make_pipeline(ct, LogisticRegression(C=1, random_state=123, solver='lbfgs'))

X_cols = ['CryoSleep', 'Destination', 'Age', 'VIP']

# 학습, 검증셋을 만듭니다.
df_train = df_space3.loc[df_space3.index % 4 != 0]
df_test = df_space3.loc[df_space3.index % 4 == 0]

# HomePlanet 별로 모델을 만들고 학습합니다.
s_lr = df_train.groupby('HomePlanet')[X_cols + ['Transported']].apply(
    lambda x: build_model().fit(x[X_cols], x['Transported'])
)
s_lr

HomePlanet
          (ColumnTransformer(n_jobs=None, remainder='dro...
Earth     (ColumnTransformer(n_jobs=None, remainder='dro...
Europa    (ColumnTransformer(n_jobs=None, remainder='dro...
Mars      (ColumnTransformer(n_jobs=None, remainder='dro...
dtype: object

In [78]:
# roc_auc_score는 예측 label을 입력 받는게 아니라, score를 입력을 받습니다. target label일 거란 score를 입력을 받아서 계산을 합니다.
# binary class일 경우에는 positive class에 대한 크기가 n인 1차원 array 이 고 
# multi class일 경우에는 n x m 2차원 array입니다.(n은 관측수, m은 class의 종류수)
from sklearn.metrics import roc_auc_score

# 그룹별 roc_auc_score를 구합니다.
# Apply는 x.name에 그룹값이 옵니다. 이를 활용하여 Positive일 확률을 구합니다.
s_roc = df_test.groupby('HomePlanet')[X_cols + ['Transported']].apply(
    lambda x: roc_auc_score(x['Transported'], s_lr.loc[x.name].predict_proba(x[X_cols])[:, 1])
)
s_roc

HomePlanet
          0.797222
Earth     0.686698
Europa    0.837234
Mars      0.861424
dtype: float64

In [79]:
# HomePlanet 별로 구분하지 않고 roc_auc_score를 구해봅니다.
df_roc = df_test.groupby('HomePlanet')[X_cols + ['Transported']].apply(
    lambda x: pd.DataFrame({'prob':s_lr.loc[x.name].predict_proba(x[X_cols])[:, 1]}, index=x.index)
)
roc_auc_score(df_test['Transported'] ,df_roc['prob'].sort_index())

0.7929791154475884

## filter 

filter에는 DataFrame이 전달되고 반환은 단일 불리언 값을 합니다. 

넘겨준 불리언 값이 True이면 Combine시 전달받은 DataFrame을 포함하고 그렇지 않으면 제외합니다.

**pd.core.groupby.DataFrameGroupBy.filter(self, func, dropna=True)**

func: function

    각각의 그룹화한 DataFrame을 입력을 받고 포함여부를 반환해주는 함수입니다.
    
dropna: 
    
    groupby에서 설정한 그룹 기준값에 결측치가 있으면 제외할지를 정합니다. 
    (가이드상에는 이렇게 나왔지만, 시험장 버젼에서는 설정대로 동작하지 않습니다)

In [80]:
def examine_filter(x):
    print(type(x), len(x), x.shape)
    return True # 무조건 포험합니다.

In [81]:
df_space.groupby('HomePlanet').filter(examine_filter)

<class 'pandas.core.frame.DataFrame'> 4602 (4602, 14)
<class 'pandas.core.frame.DataFrame'> 2131 (2131, 14)
<class 'pandas.core.frame.DataFrame'> 1759 (1759, 14)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [82]:
df_space.groupby('HomePlanet').filter(examine_filter, dropna=False)

<class 'pandas.core.frame.DataFrame'> 4602 (4602, 14)
<class 'pandas.core.frame.DataFrame'> 2131 (2131, 14)
<class 'pandas.core.frame.DataFrame'> 1759 (1759, 14)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0.0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0.0
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1.0
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0.0


In [83]:
df_space2.groupby('HomePlanet').filter(examine_filter)

<class 'pandas.core.frame.DataFrame'> 201 (201, 18)
<class 'pandas.core.frame.DataFrame'> 4602 (4602, 18)
<class 'pandas.core.frame.DataFrame'> 2131 (2131, 18)
<class 'pandas.core.frame.DataFrame'> 1759 (1759, 18)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,RoomService_std,RoomService_fn,ShoppingMall_fn,RoomService_rnk
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,-0.183573,0.0,0.0,2095.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,-0.073255,109.0,25.0,893.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,-0.129221,43.0,0.0,209.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,-0.183573,0.0,371.0,2095.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0.435386,303.0,151.0,660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,-0.183573,0.0,0.0,2095.0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,-0.359038,0.0,0.0,4499.0
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,-0.359038,0.0,1872.0,4499.0
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,-0.183573,0.0,0.0,2095.0


* Case 1: df_space2에서 PassengerId를 _ 를 기준으로 나누고, 앞에 있는 4자리 문자가 동일한 행의 수가 3개 이상을 행만을 출력하세요.

In [84]:
df_space2.pipe(
    lambda x: pd.concat([x, x['PassengerId'].str.split('_', expand=True).rename(columns={0: 'pid_1', 1:'pid_2'})], axis=1)
).groupby('pid_1').filter(
    lambda x: len(x) >= 3
)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,RoomService_std,RoomService_fn,ShoppingMall_fn,RoomService_rnk,pid_1,pid_2
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True,-0.183573,0.0,0.00000,2095.0,0008,01
10,0008_02,Europa,True,B/1/P,TRAPPIST-1e,34.0,False,0.0,0.0,NaN,0.0,0.0,Altardr Flatic,True,-0.183573,0.0,151.13282,2095.0,0008,02
11,0008_03,Europa,False,B/1/P,55 Cancri e,45.0,False,39.0,7295.0,589.0,110.0,124.0,Wezena Flatic,True,-0.134277,39.0,589.00000,212.0,0008,03
21,0020_01,Earth,True,E/0/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Almary Brantuarez,False,-0.359038,0.0,0.00000,4499.0,0020,01
22,0020_02,Earth,True,E/0/S,55 Cancri e,49.0,False,0.0,0.0,0.0,0.0,0.0,Glendy Brantuarez,False,-0.359038,0.0,0.00000,4499.0,0020,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8654,9231_02,Mars,False,F/1888/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Walls Paie,True,-0.583464,0.0,0.00000,1723.0,9231,02
8655,9231_03,Mars,False,E/592/P,TRAPPIST-1e,22.0,False,0.0,0.0,0.0,0.0,0.0,Cus Paie,False,-0.583464,0.0,0.00000,1723.0,9231,03
8685,9275_01,Europa,False,A/97/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Polaton Conable,True,-0.183573,0.0,0.00000,2095.0,9275,01
8686,9275_02,Europa,False,A/97/P,TRAPPIST-1e,32.0,False,1.0,1146.0,0.0,50.0,34.0,Diram Conable,False,-0.182309,1.0,0.00000,378.0,9275,02


* Case 2: df_space2에서 PassengerId를 _ 를 기준으로 나누고, 앞에 있는 4자리 문자가 동일한 행의 수가 3개 이상이고 CryoSleep이 모두 True인 행들만 반환하세요.

In [85]:
df_space2.pipe(
    lambda x: pd.concat([x, x['PassengerId'].str.split('_', expand=True).rename(columns={0: 'pid_1', 1:'pid_2'})], axis=1)
).groupby('pid_1').filter(
    lambda x: len(x) >= 3 and (x['CryoSleep'] == True).mean() == 1
)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,RoomService_std,RoomService_fn,ShoppingMall_fn,RoomService_rnk,pid_1,pid_2
43,0044_01,Earth,True,G/3/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Jodye Coopelandez,False,-0.359038,0.0,0.0,4499.0,0044,01
44,0044_02,Earth,True,G/3/P,55 Cancri e,4.0,False,0.0,0.0,0.0,0.0,0.0,Kayne Coopelandez,True,-0.359038,0.0,0.0,4499.0,0044,02
45,0044_03,Earth,True,G/3/P,PSO J318.5-22,21.0,False,0.0,0.0,0.0,0.0,0.0,Cassa Coopelandez,True,-0.359038,0.0,0.0,4499.0,0044,03
414,0453_01,Europa,True,B/14/S,TRAPPIST-1e,19.0,False,0.0,0.0,0.0,0.0,0.0,Aldun Taptiritty,True,-0.183573,0.0,0.0,2095.0,0453,01
415,0453_02,Europa,True,NaN,55 Cancri e,34.0,False,0.0,0.0,0.0,0.0,0.0,Minopus Taptiritty,True,-0.183573,0.0,0.0,2095.0,0453,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7710,8226_02,Mars,True,F/1572/S,TRAPPIST-1e,37.0,False,0.0,0.0,0.0,0.0,0.0,Guark Mille,True,-0.583464,0.0,0.0,1723.0,8226,02
7711,8226_03,Mars,True,F/1572/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Crunde Mille,True,-0.583464,0.0,0.0,1723.0,8226,03
8628,9205_01,Europa,True,B/300/P,55 Cancri e,37.0,False,0.0,0.0,0.0,0.0,0.0,Thaldah Brakeng,True,-0.183573,0.0,0.0,2095.0,9205,01
8629,9205_02,Europa,True,B/300/P,TRAPPIST-1e,15.0,False,0.0,0.0,0.0,0.0,0.0,NaN,True,-0.183573,0.0,0.0,2095.0,9205,02


# Pivot Table / Cross Table

pivot table / cross table 은 groupby의 aggregation을 2차원 상으로 확장한 것이라고 볼 수 있습니다. 

즉, 그룹의 기준을 index와 column 두 영역에 설정하여 index와 columns에 동시에 해당이 되는 데이터 요소들로 그룹화 해줍니다.

그리고 집계함수 aggregation 함수를 통해 묶인 데이터 요소들을 하나의 값으로 묶어 줍니다.

**pd.DataFame.pivot_table(self, values=None, index=None,columns=None,
                        fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)**

values: 컬럼명
    집계 대상 값

index: 컬럼명, list

    index 영역에서의 그룹 기준
    
columns: 컬럼명, list

    column 영역에서의 그룹 기준
    
aggfunc: str, function, list

    집계 함수명

fill_value: NA가 있을 때 대체할 값

margins: 소계 포함

dropna: 결측 제거 여부

**pd.crosstab(index, columns, values=None, rownames=None, colsnames=None,
                aggfunc=None, margins=False, margins_name='All', dropna=True, normalize=False)**

index; array_like, Series, list of Series

    index의 그룹 기준이 되는 데이터
    
columns: array_like, Series, list of Series
     
    column의 그룹 기준이 되는 데이터
    
values: array_like, Series, list of Series

    집계를 내고자 하는 데이터
    
rownames: list

    행(인덱스)의 이름
    
colsnames: list

    열(컬럼)의 이름

aggfunc: function

    집계 함수
    
margins: False

    행 / 컬럼에 소계 포함 여부
    
normalize:

     values, aggfunc 가 설정이  없을 때 유효하고, 출현 빈도가 아니라 출현 비율을 계산합니다.

### Case 1

df_space에서 HomePlanet과 Destination에 결측이 하나라도 있으면 제외하고, HomePlanet와 Destination 별 Transported가 True인 카운트를 아래 DataFrame 처럼 구하세요. 컬럼과 인덱스의 순서는 일치하지 않아도 됩니다.

|  |   |TRAPPIST-1e|PSO J318.5-22|55 Cancri e|
|---|----|----|----|----|
|Earth|OO|OO|OO|OO|OO|
|Europa|OO|OO|OO|OO|OO|
|Mars|OO|OO|OO|OO|OO|

In [86]:
# pd.DataFrame.pivot_table 이용
df_space.loc[df_space[['HomePlanet', 'Destination']].notna().all(axis=1)]\
        .pivot_table(index='HomePlanet', columns='Destination', values='Transported', aggfunc='sum')

Destination,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
HomePlanet,,,
Earth,348.0,355.0,1207.0
Europa,611.0,14.0,755.0
Mars,118.0,22.0,755.0


In [87]:
# pd.crosstab 을 이용

df_tmp = df_space.loc[df_space[['HomePlanet', 'Destination']].notna().all(axis=1)]
pd.crosstab(index=df_tmp['HomePlanet'], columns=df_tmp['Destination'], values=df_tmp['Transported'], aggfunc='sum')

Destination,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
HomePlanet,,,
Earth,348.0,355.0,1207.0
Europa,611.0,14.0,755.0
Mars,118.0,22.0,755.0


In [88]:
# margin=True로 하면 합게가 나옵니다

df_space.loc[df_space[['HomePlanet', 'Destination']].notna().all(axis=1)]\
        .pivot_table(index='HomePlanet', columns='Destination', values='Transported', aggfunc='sum', margins=True)

Destination,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,All
HomePlanet,,,,
Earth,348.0,355.0,1207.0,1910.0
Europa,611.0,14.0,755.0,1380.0
Mars,118.0,22.0,755.0,895.0
All,1077.0,391.0,2717.0,4185.0


### Case 2

HomePlanet={'Earth', 'Europa', 'Mars'}

Destination={'55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e'}

$Chi_2=\sum_{i\in{HomePlanet}, j\in{Destination}}\frac{(E_{ij} - O_{ij})^2}{E_{ij}}$ 

   $O_{ij}$는 i HomePlanet과 j 목적지의 실제 관측수, $E_{ij}$는 i HomePlanet과 j Destination에 대한 기대 관측수 입니다.
   
HomePlanet과 Destination이 모두 결측이 아닌 관측만을 사용합니다.

scipy.stats.chi2_contingency 함수를 쓰면 계산할 수 있지만, 데이터처리 연습차원에서 직접구해봅니다.

In [89]:
# 관측수를 구합니다.

df_tmp = df_space.loc[df_space[['HomePlanet', 'Destination']].notna().all(axis=1)]
df_o = pd.crosstab(index=df_tmp['HomePlanet'], columns=df_tmp['Destination'], margins=True)
df_o

Destination,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,All
HomePlanet,,,,
Earth,690,712,3101,4503
Europa,886,19,1189,2094
Mars,193,49,1475,1717
All,1769,780,5765,8314


In [90]:
# pop은 DataFrame에서 컬럼하나를 뽑으면서(연산후 데이터프레임에서 제거됩니다) 해당 Series를 반환합니다.
# HomePlanet 별 비율을 뽑기 위해 소계를 가져옵니다.
s_sum_homeplanet = df_o.pop('All')
# 총계를 뽑아냅니다.
total = s_sum_homeplanet.pop('All')
# HomePlanet 별로 비율을 구합니다.
s_ratio_homeplanet = s_sum_homeplanet / total
# 이 비율을 행성별 총계에 곱하면 각각의 기대 빈도수가 나옵니다.
s_ratio_homeplanet

HomePlanet
Earth     0.541617
Europa    0.251864
Mars      0.206519
Name: All, dtype: float64

In [91]:
# Desination 별합을 가져옵니다.
df_o = df_o.T # Destination 별 소계를 얻기 위해 Transpose합니다.
s_sum_destination = df_o.pop('All')
df_o = df_o.T # 다시 원상 복귀하여 원래의 형태로 바꿉니다.
df_e = s_ratio_homeplanet.to_frame().dot(s_sum_destination.to_frame().T) # Series를 DataFrame로 바꾸어 행렬곱을 합니다.
df_e

Destination,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
HomePlanet,,,
Earth,958.119678,422.460909,3122.419413
Europa,445.547991,196.454174,1451.997835
Mars,365.332331,161.084917,1190.582752


In [92]:
chi_2 = np.sum(np.sum(np.square(df_e - df_o) / df_e))
chi_2

1144.18502191443

In [93]:
# chi2 test까지 해봅니다.

from scipy.stats import chi2

chi2.sf(chi_2, df=4), (1 - chi2.cdf(chi_2, df=4)) # sf 는 1 - chi2.cdf이지만, 때때로 더 정확합니다.

(2.0026455334311217e-246, 0.0)

In [94]:
# 위 과정을 chi2_contingency를 사용하면 
from scipy.stats import chi2_contingency

# 반환깂은 (검정통계량, pvalue, 자유도, Expectation Table)입니다
chi2_contingency(pd.crosstab(index=df_tmp['HomePlanet'], columns=df_tmp['Destination']))

(1144.18502191443,
 2.0026455334311217e-246,
 4,
 array([[ 958.11967765,  422.46090931, 3122.41941304],
        [ 445.54799134,  196.45417368, 1451.99783498],
        [ 365.33233101,  161.08491701, 1190.58275198]]))

### Case 3

df_space에서 HomePlanet, Destination, Cabin, Transported의 결측치가 하나라도 있으면 제외하고 네 개의 컬럼만을 뽑아 space4 데이터프레임을 만듭니다. 

아래 과정은 df_space4 데이터프레임을 대상으로 진행합니다.

Cabin을 '/'로 구분하여 세 개의 파생변수, section, lineno, side 컬럼 세 개의 컬럼을 만들어 space4에 추가합니다.

동일 side에서  HomePlanet, Destination 으로 구분했을 때, 

그룹별 전체 인원 대비 Trasnported가 True의 인원의 비율아래와 같은 구성의 테이블로 만듭니다.

|    |side|P|S|
|----|-----|---|---|
|HomePlanet|Destination|||
|Earth|55 cancri e| 0.450617|0.553977|
|     |PSO J318.5-22| 0.485795|0.520349|
....

In [95]:
# df_space4 데이터프레임을 만듭니다.
df_space4 = df_space[['HomePlanet', 'Destination', 'Cabin', 'Transported']].pipe(lambda x: x.loc[x.notna().all(axis=1)])
df_space4[['section', 'lineno', 'side']] = df_space4['Cabin'].str.split('/', expand=True)
df_space4.head()

,HomePlanet,Destination,Cabin,Transported,section,lineno,side
0,Europa,TRAPPIST-1e,B/0/P,False,B,0,P
1,Earth,TRAPPIST-1e,F/0/S,True,F,0,S
2,Europa,TRAPPIST-1e,A/0/S,False,A,0,S
3,Europa,TRAPPIST-1e,A/0/S,False,A,0,S
4,Earth,TRAPPIST-1e,F/1/S,True,F,1,S


In [96]:
df_space4.pivot_table(index=['HomePlanet', 'Destination'], columns='side', values='Transported', aggfunc='mean')

side                             P         S
HomePlanet Destination                      
Earth      55 Cancri e    0.450617  0.553977
           PSO J318.5-22  0.485795  0.520349
           TRAPPIST-1e    0.323415  0.457105
Europa     55 Cancri e    0.643564  0.726667
           PSO J318.5-22  0.625000  0.900000
           TRAPPIST-1e    0.569231  0.691108
Mars       55 Cancri e    0.598039  0.629213
           PSO J318.5-22  0.392857  0.550000
           TRAPPIST-1e    0.491935  0.532189

### Case 4

Group와 pivot 테이블을 아우르는 예를 만들어 봅니다. 

같은 HomePlanet에 대해서 Destination과 side가 서로 독립인지, chi2 독립성 검정을 통해서, 각각의 HomePlanet 별로 검정을 하여, 

검정통계량은 chi2, pvalue는 pval 컬럼으로  아래와 같은 데이터프레임을 만드세요 

|HomePlanet|chi2|pval|
|---|----|----|
|Earth|1.949544|0.377278|
...

In [96]:
from scipy.stats import chi2_contingency

def chi2_test(x):
    """
    chi2 독립성 검정을 수행합니다.
    """
    chi2_result = chi2_contingency(x)
    return chi2_result[0], chi2_result[1] # chi2, pval
df_space4.groupby('HomePlanet').apply(
    lambda x: pd.DataFrame(
                    [chi2_test(x.pivot_table(index='Destination', columns='side', aggfunc='size'))], # 빈도수 테이블을 만들어 검정을 합니다.
                    columns=['chi2', 'pvalue']
                )
).reset_index(-1, drop=True)

,chi2,pvalue
HomePlanet,,
Earth,1.949544,0.377278
Europa,1.270739,0.529740
Mars,1.033293,0.596518
